In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
with open('intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [3]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [4]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [5]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.1531 - loss: 2.0794
Epoch 2/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.1531 - loss: 2.07878
Epoch 3/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.1531 - loss: 2.0781  
Epoch 4/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1531 - loss: 2.0778
Epoch 5/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.2756 - loss: 2.07717
Epoch 6/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2039 - loss: 2.0767 
Epoch 7/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.1531 - loss: 2.07656
Epoch 8/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1531 - loss: 2.0762
Epoch 9/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1531 - loss: 2.0757
Epoch 10/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.1427 - loss: 2.07606
Epoch 11/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1427 - loss: 2.0758
Epoch 12/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1531 - loss: 2.075

In [7]:
# to save the trained model
model.save("chat_model.keras")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model.keras')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
1/1r:  ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step
ChatBot: I'm Joana, an Artificial Intelligent bot
1/1r:  ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
ChatBot: I'm Joana, an Artificial Intelligent bot
1/1r:  ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
ChatBot: I'm Joana, an Artificial Intelligent bot
1/1r:  ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
ChatBot: I.m Joana, your bot assistant
1/1r:  ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
ChatBot: I.m Joana, your bot assistant
1/1r:  ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
ChatBot: I'm Joana, an Artificial Intelligent bot
1/1r:  ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
ChatBot: I'm Joana, an Artificial Intelligent bot
1/1r:  ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
ChatBot: I'm Joana, an Artif